In [3]:
# pour naviguer dans les répertoires
import os 
# pour manipuler les DataFrames
import pandas as pd
# pour ouvrir et inspecter les images
from PIL import Image

import numpy as np

# -----------------------------
# Parcours du dataset et collecte des métadonnées
# -----------------------------

# Répertoire de base où sont stockées les variantes d’images
base_dir = "../dataset/plantvillage/data/plantvillage_5images"
# liste qui accueillera un dict par image
data = []


# On parcourt les variantes/sous-dossiers "color", "grayscale", "segmented"
for variant in os.listdir(base_dir):
    variant_path = os.path.join(base_dir, variant)
    
    # Vérifie que c'est bien un dossier (pas un fichier)
    if os.path.isdir(variant_path):
        # Puis on parcourt chaque dossier d’espèce+disease
        for folder_name in os.listdir(variant_path):
            folder_path = os.path.join(variant_path, folder_name)
            
            if os.path.isdir(folder_path):
                # Extraire l’espèce et la maladie depuis le nom de dossier
                # Exemple : "Apple_scab__Powdery_mildew"
                if '__' in folder_name:
                    species, disease = folder_name.split('__', 1)
                else:
                    species, disease = folder_name, 'unknown'

                # Parcours de toutes les images dans ce dossier
                for image_name in os.listdir(folder_path):
                    image_path = os.path.join(folder_path, image_name)
                    
                    try:
                        # Ouverture sécurisée de l’image
                        with Image.open(image_path) as img:
                            # conversion systématique en RGB
                            img = img.convert('RGB')

                            # dimensions en pixels
                            width, height = img.size
                            # mode de couleur, ex RGB, grayscale,..# sera toujours 'RGB' après conversion
                            mode = img.mode  

                            # 2) Convertit en tableau NumPy et calcule mean/std par canal
                            arr = np.array(img)  # shape = (H, W, 3)
                            mean_R, mean_G, mean_B = arr.mean(axis=(0,1)).tolist()
                            std_R,  std_G,  std_B  = arr.std(axis=(0,1)).tolist()

                    except Exception as e:
                        # En cas d’erreur (fichier corrompu…), on l’affiche et on passe
                        print(f"Erreur avec {image_path} : {e}")
                        continue

                    # On stocke toutes les infos extraites dans un dict
                    data.append({
                        'image_path': image_path.replace('\\', '/'),
                        'variant': variant,  # color, grayscale ou segmented
                        'species': species,  # nom de l’espèce
                        'disease': disease, # type de maladie (ou 'unknown')
                        'width': width, # largeur
                        'height': height, # hauteur
                        'mode': mode,  # ex. 'RGB' 
                         # colonnes couleur globale
                        #'mean_R': mean_R, 'mean_G': mean_G, 'mean_B': mean_B,
                        #'std_R':  std_R,  'std_G':  std_G,  'std_B':  std_B
                    })

# Création du DataFrame pandas à partir de la liste de dicts
df = pd.DataFrame(data)
# Affichage des premières lignes pour vérifier la structure
print(df.head())
# Liste des colonnes disponibles dans le DF
print("\nColonnes disponibles :", df.columns.tolist())
# Statistiques descriptives sur les dimensions
print("\nDimensions des images :")
print(df[['width', 'height']].describe())

# Compte des différents modes d’image (RGB, L pour grayscale, etc.)
print("\nTypes d’image :", df['mode'].value_counts())
# Répartition du nombre d’images par variante (color, grayscale, segmented)
print("\nRépartition des variantes :", df['variant'].value_counts())

# Distribution des espèces
print(df['species'].value_counts())
# Distribution des maladies
print(df['disease'].value_counts())

                                          image_path    variant species  \
0  ../dataset/plantvillage/data/plantvillage_5ima...  segmented  Tomato   
1  ../dataset/plantvillage/data/plantvillage_5ima...  segmented  Tomato   
2  ../dataset/plantvillage/data/plantvillage_5ima...  segmented  Tomato   
3  ../dataset/plantvillage/data/plantvillage_5ima...  segmented  Tomato   
4  ../dataset/plantvillage/data/plantvillage_5ima...  segmented  Tomato   

                disease  width  height mode  
0  _Tomato_mosaic_virus    256     256  RGB  
1  _Tomato_mosaic_virus    256     256  RGB  
2  _Tomato_mosaic_virus    256     256  RGB  
3  _Tomato_mosaic_virus    256     256  RGB  
4  _Tomato_mosaic_virus    256     256  RGB  

Colonnes disponibles : ['image_path', 'variant', 'species', 'disease', 'width', 'height', 'mode']

Dimensions des images :
       width  height
count  570.0   570.0
mean   256.0   256.0
std      0.0     0.0
min    256.0   256.0
25%    256.0   256.0
50%    256.0   256.0
75

In [4]:
# vérif rapide
display(df.head())
print("Colonnes :", df.columns.tolist())

,image_path,variant,species,disease,width,height,mode
0,../dataset/plantvillage/data/plantvillage_5ima...,segmented,Tomato,_Tomato_mosaic_virus,256,256,RGB
1,../dataset/plantvillage/data/plantvillage_5ima...,segmented,Tomato,_Tomato_mosaic_virus,256,256,RGB
2,../dataset/plantvillage/data/plantvillage_5ima...,segmented,Tomato,_Tomato_mosaic_virus,256,256,RGB
3,../dataset/plantvillage/data/plantvillage_5ima...,segmented,Tomato,_Tomato_mosaic_virus,256,256,RGB
4,../dataset/plantvillage/data/plantvillage_5ima...,segmented,Tomato,_Tomato_mosaic_virus,256,256,RGB


Colonnes : ['image_path', 'variant', 'species', 'disease', 'width', 'height', 'mode']


In [6]:
from sklearn.model_selection import train_test_split

# ---------------------------------------
# 2) Séparation X / y et split
# ---------------------------------------
X     = df.drop(columns=["species","image_path"], axis=1)
y     = df["species"]

# split avant tout fit
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=66, stratify=y
)


In [7]:
# ---------------------------------------
# 3) Sélection des colonnes num & cat
# ---------------------------------------
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object"]).columns.tolist()

In [8]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# ---------------------------------------
# 4) Pré-traitement séparé pour train et test
# ---------------------------------------
# 4a) scaler sur les numériques
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train[num_cols])
X_test_num_scaled  = scaler.transform(X_test[num_cols])

# remettre dans des DataFrame pour concat
X_train_num_df = pd.DataFrame(
    X_train_num_scaled,
    index=X_train.index,
    columns=num_cols
)
X_test_num_df = pd.DataFrame(
    X_test_num_scaled,
    index=X_test.index,
    columns=num_cols
)

# 4b) one-hot encoding des catégorielles
encoder = OneHotEncoder(
    drop="first",
    handle_unknown="ignore",
    sparse_output=False
)
X_train_cat = encoder.fit_transform(X_train[cat_cols])
X_test_cat  = encoder.transform(  X_test[cat_cols])

# noms des nouvelles colonnes
cat_names = encoder.get_feature_names_out(cat_cols)

X_train_cat_df = pd.DataFrame(
    X_train_cat,
    index=X_train.index,
    columns=cat_names
)
X_test_cat_df = pd.DataFrame(
    X_test_cat,
    index=X_test.index,
    columns=cat_names
)

# concaténation finale
X_train_proc = pd.concat([X_train_num_df, X_train_cat_df], axis=1)
X_test_proc  = pd.concat([X_test_num_df,  X_test_cat_df],  axis=1)

In [10]:
from sklearn.svm import SVC
# ---------------------------------------
# 5) Entraînement du SVM
# ---------------------------------------
clf = SVC(kernel="rbf")
clf.fit(X_train_proc, y_train)

print("Score train :", clf.score(X_train_proc, y_train))
print("Score test :",  clf.score(X_test_proc,  y_test))

Score train : 0.5614035087719298
Score test : 0.5175438596491229


In [12]:
print("Avec seulement 5 images par classe : ~ 0.55 en train et ~ 0.52 en test. Et Score ≈ 0.5 = presque aléatoire")
print(" tester en augmentant la taille du dataset")

Avec seulement 5 images par classe : ~ 0.55 en train et ~ 0.52 en test. Et Score ≈ 0.5 = presque aléatoire
 tester en augmentant la taille du dataset
